In [1]:
from dataclasses import dataclass
import itertools
import logging
import random
import math
import numpy as np
import json
import pickle
import time
import torch
import sys
import yaml
import seaborn as sns
import matplotlib.pyplot as plt
from probe_utils import *
from plot_utils import *
from tqdm import tqdm

from omegaconf import OmegaConf
from pathlib import Path
from torch import nn, Tensor
from torch.nn import functional as F
from typing import List, Optional, Tuple
import os
# os.chdir("/data/tianyu_guo/birth")
from data import DataArgs, Dataset, iterate_batches, make_dataset
from ihead_full_model import *

%load_ext autoreload
%autoreload 2


In [2]:
torch.cuda.set_device(0)

In [3]:
run_path_server = "/data/tianyu/birth/gens/pre-iclr/dynamics/dormant_copy_k3_bos1"
# run_path_server = "/data/tianyu/birth/gens/pre-iclr/dynamics/dormant_copy_simplified"
# run_path_server2="/data/tianyu_guo/birth/gens/special/dormant_copy_2"
model, cfg, x, y, ds, optim = load_model(run_path_local="/Users/guotianyu/GitHub/birth/gens/special/markov", run_path_server=run_path_server, bos_num=1, train_steps=10000, delim=0, n_layers=3, n_heads=1, no_attn_norm=(), no_ffn_norm=(), no_attn=(), no_ffn=(), linear_ffn=(), with_data=True, with_optim=True, data_path_local="/Users/guotianyu/GitHub/birth/data", data_path_server="/data/tianyu/birth/data")
hook = forward_hook([], '')
pred, outputs_list = model.modified_forward_with_hook(x, hook)
probs = get_oracle_predicts(x, ds)
triggers_pos = ds.get_triggers_pos(x)
risk = get_risk(probs, pred, predict_in_logits=True, triggers_pos=triggers_pos)
risk[1:3]

/data/tianyu/miniconda3/envs/nanogpt/lib/python3.10/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


(tensor(5.9845e-05, grad_fn=<MeanBackward0>),
 tensor(2.2292, grad_fn=<MeanBackward0>))

In [4]:
outputs_list[0]['input'][0, 0, :10], outputs_list[0]['attn_output_add_res'][0, 0, :10], outputs_list[0]['output'][0, :10, :].norm(dim=-1), outputs_list[1]['value_states'][0, 0, :10, :].norm(dim=-1), outputs_list[1]['attn_logits'][0, 0, :10, 0]

(tensor([ 0.1626,  0.3461, -0.2837,  2.1683, -0.5298,  0.3862,  0.4549,  0.7601,
         -0.2890, -0.4319], grad_fn=<SliceBackward0>),
 tensor([  2.3343,  -0.9709,   0.3139,   5.2199,  -3.0113,  -8.2812,  -3.9360,
           1.4251, -10.3032,  -8.6213], grad_fn=<SliceBackward0>),
 tensor([140.7036,  27.3864,  17.4664,  27.2720,  32.0737,  31.4242,  31.6992,
          16.8821,  32.4395,  32.2114], grad_fn=<LinalgVectorNormBackward0>),
 tensor([ 2.3901,  6.6247,  6.4160,  6.2314, 11.2944, 10.1937, 11.4058,  7.3493,
         11.8356, 12.5429], grad_fn=<LinalgVectorNormBackward0>),
 tensor([-13.0613,   8.5810,  -1.0609,   7.2337,   5.1180,   4.1996,   5.6518,
          -2.7330,   5.5044,   5.4849], grad_fn=<SelectBackward0>))

In [5]:
model.train()
pred = model(x)
loss = F.cross_entropy(pred.flatten(0, 1), y.flatten(0, 1))
loss.backward()
## get the gradient of the loss with respect to the output of the last layer
grads = {}
for name, param in model.named_parameters():
    grads[name] = param.grad

In [6]:
grads

{'tok_embeddings.weight': tensor([[-2.8835e-06, -1.3700e-06, -1.0619e-06,  ...,  2.4604e-06,
           2.3522e-06, -1.3192e-06],
         [ 1.4683e-06, -1.6403e-07,  1.9367e-07,  ...,  2.2831e-07,
          -1.0986e-06,  4.0312e-07],
         [-1.5620e-07, -6.0497e-08, -1.8145e-06,  ...,  4.1030e-07,
           6.0130e-07,  8.7628e-07],
         ...,
         [ 1.0631e-06,  2.4995e-06,  3.5275e-07,  ...,  5.8857e-07,
          -6.4554e-07, -1.0932e-06],
         [-1.4841e-06,  1.4228e-06, -1.3573e-06,  ..., -1.6052e-07,
          -1.0588e-06, -3.3163e-07],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00]]),
 'pos_embeddings.weight': tensor([[-1.4841e-06,  1.4228e-06, -1.3573e-06,  ..., -1.6052e-07,
          -1.0588e-06, -3.3162e-07],
         [-1.3512e-06, -1.2674e-06, -8.3697e-07,  ...,  6.7605e-07,
          -1.0531e-06, -1.7553e-07],
         [-8.9438e-07, -9.9600e-07,  7.4069e-07,  ...,  1.0507e-06,
          -7.6084e-07, -1.